In [2]:
# %pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 47.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import numpy as np

# Create a simple sample dataset
np.random.seed(42)  # For reproducibility

num_samples = 1000
num_features = 20
num_classes = 5

# Generate random data
X = np.random.randn(num_samples, num_features)
# Generate random labels from 0 to 4 (5 classes)
y = np.random.randint(0, num_classes, num_samples)

# Alternate y so the result can easily be verified.
mask = X[:, 0] > 1
y[mask] = 1
y[~mask] = np.where(y[~mask] == 1, 0, y[~mask])


class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create datasets and loaders
train_dataset = SimpleDataset(X_train, y_train)
test_dataset = SimpleDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the neural network
class SimpleNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_dim = num_features
num_classes = num_classes

model = SimpleNN(input_dim=input_dim, num_classes=num_classes)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping parameters
patience = 3
best_loss = float('inf')
patience_counter = 0

# Training loop with early stopping
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=50):
    global best_loss, patience_counter
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # Zero the parameter gradients
            optimizer.zero_grad()
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        running_loss /= len(train_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss:.4f}')

        # Validation loss for early stopping
        val_loss = validate_model(model, test_loader)
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f'Early stopping on epoch {epoch + 1}')
            break

# Validation function
def validate_model(model, test_loader):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(test_loader)
    print(f'Validation Loss: {val_loss:.4f}')
    return val_loss

# Train the model
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=50)

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Epoch [1/50], Loss: 1.5931
Validation Loss: 1.5857
Epoch [2/50], Loss: 1.5627
Validation Loss: 1.5683
Epoch [3/50], Loss: 1.5313
Validation Loss: 1.5370
Epoch [4/50], Loss: 1.4804
Validation Loss: 1.4830
Epoch [5/50], Loss: 1.3993
Validation Loss: 1.4150
Epoch [6/50], Loss: 1.3122
Validation Loss: 1.3560
Epoch [7/50], Loss: 1.2448
Validation Loss: 1.3206
Epoch [8/50], Loss: 1.1996
Validation Loss: 1.3027
Epoch [9/50], Loss: 1.1665
Validation Loss: 1.2929
Epoch [10/50], Loss: 1.1447
Validation Loss: 1.2906
Epoch [11/50], Loss: 1.1220
Validation Loss: 1.2865
Epoch [12/50], Loss: 1.1023
Validation Loss: 1.2848
Epoch [13/50], Loss: 1.0877
Validation Loss: 1.2913
Epoch [14/50], Loss: 1.0706
Validation Loss: 1.2850
Epoch [15/50], Loss: 1.0580
Validation Loss: 1.2944
Early stopping on epoch 15
Accuracy on the test set: 42.00%


In [36]:
class_one = np.random.randn(10, num_features)

class_one[:,0] = np.abs(class_one[:,0]) + 1
class_one

array([[ 2.11578471,  0.02462599, -1.17783636, -0.42474439,  0.90291655,
        -1.02912968, -0.50431203,  0.07163128,  0.2468869 , -0.49929059,
         0.78686007,  0.5904523 ,  1.1398566 , -0.08160192, -0.51281881,
        -0.68263202,  0.30702752, -0.37715578, -0.56916978, -0.44828726],
       [ 1.68369049,  0.19902111, -1.41258318,  0.30301629,  0.67654876,
        -0.58087914,  1.92203694,  0.01762131, -1.56134298, -0.28124291,
         0.06265614,  0.44896756, -0.25743052, -1.01157028, -1.09498062,
        -0.65359929, -0.55777536, -1.13497416, -0.73281316,  0.65945852],
       [ 2.64841261,  0.75546171,  1.30586109,  0.97411847,  0.83313033,
        -0.33779626,  0.14091088, -0.42061934, -0.21910716,  0.0681671 ,
         0.71356615,  0.42332338, -0.16182926,  0.05553874,  0.43346367,
         0.66739444, -1.51899516, -1.07738896,  1.10731053, -0.06086957],
       [ 1.42407659, -0.26349947, -0.26370531,  0.55793778, -1.10801639,
         0.02477775,  0.24611773,  0.39626792,  

In [37]:
class_other = np.random.randn(20, num_features)
class_other

array([[ 7.36388929e-02, -7.84441919e-01,  4.31705785e-01,
        -2.31408002e-01,  9.30291709e-01, -1.13626935e-01,
        -8.61023106e-01, -1.53586467e+00, -1.86406483e+00,
         1.31250934e+00, -4.57789353e-01,  3.38574190e-01,
         1.78322989e+00, -5.62804331e-01, -6.27477484e-01,
        -1.32161752e-01,  2.94327004e-02,  1.88755135e+00,
         1.56582059e+00,  6.20875416e-01],
       [ 1.44941377e+00, -1.78219013e-01,  2.99164567e-01,
        -3.01513645e-01, -1.29036838e+00,  1.20387474e+00,
         5.31698734e-01,  1.80734734e-02, -2.46784931e-03,
        -1.50621487e+00, -9.77475333e-01,  3.36153170e-01,
        -5.29112667e-01, -2.17713041e-01, -1.04656065e-01,
        -6.82468769e-01, -9.67918320e-01, -1.61235763e+00,
        -1.46518279e-01,  6.18608548e-01],
       [-1.64363414e+00, -2.40478251e-02,  2.81143253e-01,
        -9.89580404e-01,  1.42214181e+00, -3.85715369e-01,
        -1.01259688e-01, -1.61269501e-01, -1.77077442e+00,
         1.00875207e+00, -9.5

In [41]:
# Function to perform inference
def infer(model, features):
    # Ensure the model is in evaluation mode
    model.eval()

    # Convert the numpy array to a PyTorch tensor
    features_tensor = torch.tensor(features, dtype=torch.float32)

    # Add a batch dimension, as the model expects inputs to be batched
    features_tensor = features_tensor.unsqueeze(0)

    # Perform inference
    with torch.no_grad():
        outputs = model(features_tensor)

    # Get the predicted class
    _, predicted = torch.max(outputs, 1)

    return predicted.item()


In [47]:

c1loader = DataLoader(class_one, batch_size=32, shuffle=True)

infer(model, class_other[2])

4

In [48]:
# Function to perform inference on multiple rows
def infer_multiple(model, features):
    # Ensure the model is in evaluation mode
    model.eval()

    # Convert the numpy array to a PyTorch tensor
    features_tensor = torch.tensor(features, dtype=torch.float32)

    # Perform inference
    with torch.no_grad():
        outputs = model(features_tensor)

    # Get the predicted classes in (max value, index of max value)
    _, predicted = torch.max(outputs, 1)

    return predicted.numpy()  # Convert the tensor back to a numpy array if needed


In [52]:
c1_predict = infer_multiple(model, class_one)
np.c_[c1_predict, class_one[:,0]]

array([[1.        , 2.11578471],
       [1.        , 1.68369049],
       [1.        , 2.64841261],
       [1.        , 1.42407659],
       [1.        , 2.59310622],
       [1.        , 1.33528347],
       [1.        , 1.30906026],
       [1.        , 1.38552713],
       [1.        , 1.39589811],
       [2.        , 1.22742127]])

In [24]:
mask = X[:, 0] > 1
y = np.random.randint(0, num_classes, num_samples)

y[mask] = 1
y[~mask] = np.where(y[~mask] == 1, 0, y[~mask])
np.c_[y, X[:,0]]

array([[ 0.        ,  0.49671415],
       [ 1.        ,  1.46564877],
       [ 0.        ,  0.73846658],
       ...,
       [ 3.        , -0.69193084],
       [ 0.        , -0.11676412],
       [ 0.        ,  0.96259198]])